In [2]:
import csv
import sys

def set_csv_field_limit():
    maxInt = sys.maxsize
    while True:
        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)
    return maxInt

def load_documents(doc_file):
    """
    Loads the document contents from the first file.

    :param doc_file: Path to the document file (document ID <TAB> document contents).
    :return: A dictionary {document_id: document_contents}.
    """
    # Set the field size limit first
    set_csv_field_limit()

    documents = {}
    with open(doc_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            if len(row)==0: continue
            doc_id, content = row
            documents[doc_id] = content
    return documents

docs = []
doc_file = 'meetings.tsv'
documents = load_documents(doc_file)
documents['doc_0']

"project manager: yep . soon as i get this . okay . this is our last meeting . um i 'll go ahead and go through the minutes from the previous meeting . uh and then we 'll have a , the prototype presentation . um then we will um do an evaluation . uh or we 'll see what , what we need to have under the criteria for the evaluation . then we 'll go through the finance and see if we fall within the budget . um then we 'll do the evaluation , and then we can finish up after that with um any changes that we 'll need to make , or hopefully everything will fall right in line . um let 's see , minutes from the last meeting . um we looked at uh the the trends . we had uh the fashion trends that people want a fancy look-and-feel . it was twice as important as anything else . um they liked fruit and vegetables in the new styles . um and a spongy feel . so we were talking about trying to incorporate those into our prototype . um they wanted limited buttons and simplicity . um then we looked at the u

In [3]:
import random

def load_questions_answers(qa_file):
    """
    Loads the questions and corresponding ground truth document IDs.

    :param qa_file: Path to the question-answer file (document ID <TAB> question <TAB> answer).
    :return: A list of tuples [(document_id, question, answer)].
    """
    qa_pairs = []
    with open(qa_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
            doc_id, question, answer = row
            qa_pairs.append((doc_id, question, answer))

    # random.shuffle(qa_pairs)

    return qa_pairs

qa_file = 'questions_answers.tsv'  # document ID <TAB> question <TAB> answer
qa_pairs = load_questions_answers(qa_file)
qa_pairs[0]


('doc_0',
 'how did project manager and user interface introduce the prototype of the remote control ?',
 "project manager introduced that the prototype incorporated fashion trends that people prefer fancy looking products like fruit and vegetable . after that , user interface presented the product which looked like a banana and was bright yellow except for the blue button . the style was as simple as possible in order to fit the customers ' need for simplicity . also , the product could be curved and used both-handed with advanced chips hidden inside , which seemed quite creative and identical to ipod features . in the end , industrial designer commented that the remote control could be smaller in size .")

In [4]:
my_collections = [documents[k] for k in documents.keys()]
doc_ids = list(documents.keys())
queries = [q for _, q, _ in qa_pairs]

In [5]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
index_path = RAG.index(index_name="my_index", collection=my_collections, document_ids=doc_ids, max_document_length=512, split_documents=True)

c:\Users\Admin\anaconda3\envs\colbert_new\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\anaconda3\envs\colbert_new\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--colbert-ir--colbertv2.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to 

[Dec 05, 05:42:05] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


c:\Users\Admin\anaconda3\envs\colbert_new\lib\site-packages\torch\utils\cpp_extension.py:382: UserWarning: Error checking compiler version for cl: [WinError 2] The system cannot find the file specified
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')


CalledProcessError: Command '['where', 'cl']' returned non-zero exit status 1.

In [ ]:
query = queries[0]
RAG = RAGPretrainedModel.from_index("my_index")
results = RAG.search(query)